# Mobile Games Development

In [2]:
import pandas as pd
import numpy as np
from operator import attrgetter
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

task1_reg = pd.read_csv('~/shared/problem1-reg_data.csv', sep=';')
task1_auth = pd.read_csv('~/shared/problem1-auth_data.csv', sep=';')

In [29]:
task1_reg.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [30]:
task1_reg.tail()

,reg_ts,uid
999995,1600874034,1110618
999996,1600874086,1110619
999997,1600874139,1110620
999998,1600874191,1110621
999999,1600874244,1110622


In [31]:
task1_reg.shape

(1000000, 2)

In [32]:
task1_reg.isna().sum()

reg_ts    0
uid       0
dtype: int64

In [33]:
task1_auth.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [34]:
task1_auth.tail()

,auth_ts,uid
9601008,1600874034,1110618
9601009,1600874086,1110619
9601010,1600874139,1110620
9601011,1600874191,1110621
9601012,1600874244,1110622


In [35]:
task1_auth.shape

(9601013, 2)

In [36]:
task1_auth.isna().sum()

auth_ts    0
uid        0
dtype: int64

In [37]:
# Данные о повторном входе в приложение содержат данные о регистрации,
# поэтому дальше использую только второй датафрейм (task1_auth)

In [123]:
def retention(data):
    """
    Перевожу данные о времени входа в приложение из формата unix

    Извлекаю день и месяц входа
    
    Присваиваю когорты на основе данных о первом входе в приложение для каждого пользователя
    
    Рассчитываю номер месячного периода (делю на 30)
    
    Агрегирую данные по когортам и рассчитанному номеру месячного периода
    
    Создаю сводную таблицу для когортного анализа
    
    Рассчитываю размеры когорт (первый столбец сводной таблицы)
    
    Вычисляю коэффициенты удержания (делю на размер когорты)

    """

    data.auth_ts = pd.to_datetime(data.auth_ts, unit='s')
    
    data['order_period'] = data.auth_ts.dt.to_period('M')
    data['order_period_day'] = data.auth_ts.dt.to_period('D')
    
    data['cohort'] = data.groupby('uid').\
                          auth_ts.transform('min').\
                          dt.to_period('M')
    data['cohort_day'] = data.groupby('uid').\
                              auth_ts.transform('min').\
                              dt.to_period('D')
    
    data['period_number_month'] = np.floor((data.order_period_day - data.cohort_day).apply(attrgetter('n')) / 30)
    
    df_chohort = data.groupby(['cohort', 'period_number_month']).\
                      agg(n_customers=('uid', 'nunique')).\
                      reset_index()
    
    cohort_pivot = df_chohort.pivot_table(index='cohort', columns='period_number_month', values='n_customers')
    
    cohort_size = cohort_pivot.iloc[:, 0]
    
    retention_matrix = cohort_pivot.divide(cohort_size, axis=0)
    retention_matrix = retention_matrix.drop(retention_matrix.columns[0], axis=1)
    
    return retention_matrix

In [124]:
retention(task1_auth)

period_number_month,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,248.0,249.0,250.0,251.0,252.0,253.0,254.0,255.0,256.0,257.0
cohort,,,,,,,,,,,,,,,,,,,,,
1998-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-07,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05,0.110207,0.051289,0.051289,0.039821,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06,0.107139,0.049093,0.036605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07,0.108017,0.036592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
print(retention.__doc__)


    Перевожу данные о времени входа в приложение из формата unix

    Извлекаю день и месяц входа
    
    Присваиваю когорты на основе данных о первом входе в приложение для каждого пользователя
    
    Рассчитываю номер месячного периода (делю на 30)
    
    Агрегирую данные по когортам и рассчитанному номеру месячного периода
    
    Создаю сводную таблицу для когортного анализа
    
    Рассчитываю размеры когорт (первый столбец сводной таблицы)
    
    Вычисляю коэффициенты удержания (делю на размер когорты)

    


In [41]:
task1_copy = task1_auth.copy()

In [42]:
task1_copy.auth_ts = pd.to_datetime(task1_copy.auth_ts, unit='s')

In [43]:
task1_copy['order_period'] = task1_copy.auth_ts.dt.to_period('M')
task1_copy['order_period_day'] = task1_copy.auth_ts.dt.to_period('D')

In [44]:
task1_copy['cohort'] = task1_copy.groupby('uid').\
                                  auth_ts.transform('min').\
                                  dt.to_period('M')
task1_copy['cohort_day'] = task1_copy.groupby('uid').\
                                      auth_ts.transform('min').\
                                      dt.to_period('D')

In [45]:
task1_copy['period_number_month'] = np.floor((task1_copy.order_period_day - task1_copy.cohort_day).\
                                       apply(attrgetter('n')) / 30)

In [46]:
df_chohort = task1_copy.groupby(['cohort', 'period_number_month']).\
                        agg(n_customers=('uid', 'nunique')).\
                        reset_index()

In [47]:
cohort_pivot = df_chohort.pivot_table(index='cohort', columns='period_number_month', values='n_customers')

In [48]:
cohort_size = cohort_pivot.iloc[:, 0]

In [49]:
retention_matrix = cohort_pivot.divide(cohort_size, axis=0)

In [121]:
retention_matrix = retention_matrix.drop(retention_matrix.columns[0], axis=1)

In [122]:
retention_matrix.head()

period_number_month,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,248.0,249.0,250.0,251.0,252.0,253.0,254.0,255.0,256.0,257.0
cohort,,,,,,,,,,,,,,,,,,,,,
1998-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
task2 = pd.read_csv('~/Chupa_Chups/Проект_1_Задание_2.csv', sep=';')

In [4]:
task2_copy = task2.copy()

In [5]:
task2_copy.head()

,user_id,revenue,testgroup
0,1,0,b
1,2,0,a
2,3,0,a
3,4,0,b
4,5,0,b


In [6]:
task2_copy.tail()

,user_id,revenue,testgroup
404765,404766,0,a
404766,404767,0,b
404767,404768,231,a
404768,404769,0,a
404769,404770,0,b


In [7]:
task2_copy.shape

(404770, 3)

In [8]:
task2_copy.describe()

,user_id,revenue
count,404770.000000,404770.000000
mean,202385.500000,26.083435
std,116847.178567,681.652928
min,1.000000,0.000000
25%,101193.250000,0.000000
50%,202385.500000,0.000000
75%,303577.750000,0.000000
max,404770.000000,37433.000000


In [9]:
task2_copy.isna().sum()

user_id      0
revenue      0
testgroup    0
dtype: int64

In [15]:
import pingouin as pg
from scipy import stats
from statsmodels.stats import proportion

test = task2_copy.query('testgroup=="b"')
control = task2_copy.query('testgroup=="a"')

In [18]:
test.head()

,user_id,revenue,testgroup
0,1,0,b
3,4,0,b
4,5,0,b
5,6,0,b
6,7,0,b


In [19]:
control.head()

,user_id,revenue,testgroup
1,2,0,a
2,3,0,a
9,10,0,a
10,11,0,a
11,12,0,a


In [20]:
# Анализирую ARPU
test_arpu = test.revenue.mean()
control_arpu = control.revenue.mean()

print(test_arpu, control_arpu)

26.75128659327863 25.413719736965806


In [21]:
test_arpu / control_arpu

1.052631683600699

In [22]:
pg.normality(test.revenue)

/opt/tljh/user/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


,W,pval,normal
revenue,0.065882,0.0,False


In [23]:
pg.normality(control.revenue)

/opt/tljh/user/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


,W,pval,normal
revenue,0.008877,0.0,False


In [24]:
# Использую Т-тест
pg.ttest(test.revenue, control.revenue)
# P-value > 0.05, следовательно, различие ARPU между группами не статистически значимо

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.623488,240991.130782,two-sided,0.532965,"[-2.87, 5.54]",0.001962,0.004,0.095694


In [25]:
# Анализирую ARPPU
test_pay_users = test.query('revenue>0')
control_pay_users = control.query('revenue>0')

In [26]:
test_pay_users.head()

,user_id,revenue,testgroup
160,161,3797,b
377,378,3768,b
386,387,3527,b
551,552,2038,b
913,914,3902,b


In [27]:
control_pay_users.head()

,user_id,revenue,testgroup
72,73,351,a
341,342,290,a
385,386,250,a
416,417,37324,a
712,713,255,a


In [28]:
test_arppu = test_pay_users.revenue.mean()
control_arppu = control_pay_users.revenue.mean()

print(test_arppu, control_arppu)

3003.6581717451522 2663.9984439834025


In [29]:
test_arppu / control_arppu

1.1274999722799635

In [30]:
pg.normality(test_pay_users.revenue)

,W,pval,normal
revenue,0.958312,2.491346e-22,False


In [31]:
pg.normality(control_pay_users.revenue)

,W,pval,normal
revenue,0.266792,0.0,False


In [32]:
pg.ttest(test_pay_users.revenue, control_pay_users.revenue)
# P-value > 0.05, следовательно, различие ARPPU между группами не статистически значимо

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.64463,1943.481522,two-sided,0.100208,"[-65.38, 744.7]",0.052132,0.142,0.356381


In [33]:
# Анализирую CR
test_cr = 1805 / 202667
control_cr = 1928 / 202103

print(test_cr, control_cr)

0.008906235351586593 0.009539690157988749


In [34]:
control_cr / test_cr

1.0711248671740197

In [35]:
# Использую Z-тест
paying = [1805, 1928]
common = [202667, 202103]

proportion.proportions_ztest(paying, common)
# P-value < 0.05, следовательно, различие CR между группами статистически значимо

(-2.108028495889841, 0.035028524642854865)

In [ ]:
# Вывод:
# лучший набор акционных предложений - набор контрольной группы,
# т.к. статистически значимым является только различие конверсий, где показатель контрольной группы выше на 7%

In [9]:
# Метрики для обычного события

# Метрика роста:
# - Количество новых игроков во время проведения события.

# Метрики продукта:
# - Среднее время игрового сеасна пользователей;
# - Средний прогресс игроков в событии (уровень после завершения события);
# - Количество игроков, которые получили все награды, прошли событие до конца;
# - ARPU;
# - ARPPU;
# - Конверсия в платящего пользователя;
# - Retention.


# Метрики для события с усложненной механикой (остаются прошлые метрики и добавляются новые):

# Метрики продукта:
# - Среднее количетсво откатов игроков;
# - Среднее количество уровней, на которые откатились игроки;
# - Уровень фрустрации (процент оттока после отката уровней);
# - ARPPU откатившихся игроков.